## 準備

1. KaggleのTitanic Datasetは必要なのでダウンロードしてGoogle Driveに上げる。 \
https://www.kaggle.com/c/titanic/data よりダウンロードが可能(Kaggleアカウントが必要) \

以下を`drive/MyDrive/Kaggle` 下に配置する。
- gender_submission.csv
- train.csv
- test.csv

2. ドライブをマウントする。 \
左のフォルダのアイコンからやる。

## 以下コード

### データの読み込み

In [54]:
# 必要なものをインストール
!pip install matplotlib==3.3.3
!pip install category_encoders

In [55]:
# データセットが入っているフォルダに移動
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [56]:
# 必要なライブラリをインストール
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import category_encoders as ce

In [4]:
issubmit = True  # Kaggleにサブミットするかどうか

In [57]:
# 学習データを読み込む
train = pd.read_csv('train.csv')
print(train.shape) # 形状を確認
train.head()  # 最初の5行を見る。 

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [58]:
# テストデータを読み込む
test = pd.read_csv('test.csv')
print(test.shape) # 形状を確認
test.head()  # 最初の5行を見る。 

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [59]:
## 型を確認する。
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [60]:
# 各特徴量の詳細を表示
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [61]:
# 欠損値の数を確認する。
print(train.isnull().sum())
print('-'*40)
print(test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------------------------------------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [62]:
# 各特徴量の種類数を表示
train.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

## 前処理

データがobject型のままではモデルが処理できないのでエンコーディングを行う。OneHot Encodingを用いる。 \

また、ニューラルネットワークでは欠損値をそのまま扱うことができないので補填する。今回は、欠損値は全体の平均値で補填してみる。

なお、trainとtestでラベルの相違があったら困るので一度全データを結合してからエンコーディングする。

In [63]:
## trainとtestを結合する。
df_cat = pd.concat([train, test])

In [64]:
# 欠損値を補填する。
null_columns = ['Age', 'Cabin']
df_cat.fillna(df_cat.mean(), inplace=True)

In [65]:
# object型のcolumnを定義する。(NameとTicketは種類が多すぎるので削除)
obj_columns = ['Sex', 'Cabin', 'Embarked']
# OneHotEncodingを適用する(category_encodersではOneHotEncoderという名前である。)
oe = ce.OneHotEncoder(cols=obj_columns)
df_cat_oe = oe.fit_transform(df_cat)
df_cat_oe.head()

,PassengerId,Survived,Pclass,Name,Sex_1,Sex_2,Age,SibSp,Parch,Ticket,Fare,Cabin_1,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Cabin_7,Cabin_8,Cabin_9,Cabin_10,Cabin_11,Cabin_12,Cabin_13,Cabin_14,Cabin_15,Cabin_16,Cabin_17,Cabin_18,Cabin_19,Cabin_20,Cabin_21,Cabin_22,Cabin_23,Cabin_24,Cabin_25,Cabin_26,Cabin_27,Cabin_28,Cabin_29,...,Cabin_152,Cabin_153,Cabin_154,Cabin_155,Cabin_156,Cabin_157,Cabin_158,Cabin_159,Cabin_160,Cabin_161,Cabin_162,Cabin_163,Cabin_164,Cabin_165,Cabin_166,Cabin_167,Cabin_168,Cabin_169,Cabin_170,Cabin_171,Cabin_172,Cabin_173,Cabin_174,Cabin_175,Cabin_176,Cabin_177,Cabin_178,Cabin_179,Cabin_180,Cabin_181,Cabin_182,Cabin_183,Cabin_184,Cabin_185,Cabin_186,Cabin_187,Embarked_1,Embarked_2,Embarked_3,Embarked_4
0,1,0.0,3,"Braund, Mr. Owen Harris",1,0,22.0,1,0,A/5 21171,7.2500,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,38.0,1,0,PC 17599,71.2833,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",0,1,26.0,0,0,STON/O2. 3101282,7.9250,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,35.0,1,0,113803,53.1000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,5,0.0,3,"Allen, Mr. William Henry",1,0,35.0,0,0,373450,8.0500,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## モデル構築

今回は全結合ニューラルネットワーク(MLP)を用いる。
MLPはLightGBMと比較すると精度は劣るが、別の角度からの分析が可能になるため、アンサンブルをする際のモデルの一つとしては有用である。
MLPの実装はkerasを用いるのが楽である。 \
なお、MLPでは、値のスケーリングを行う。

また、本ノートブックの最後にPytorchでの実装方法も紹介する。

In [66]:
## df_cat_oeをtrainとtestに分離する。
train = df_cat_oe[:len(train)]
test = df_cat_oe[len(train):]

In [67]:
# 学習に用いない特徴を定義する。
drop_cols = ['PassengerId', 'Survived', 'Name', 'Ticket']
# 学習データを定義する。
X = train.drop(drop_cols, axis=1)
y = train['Survived'].values
X_test = test.drop(drop_cols, axis=1)

In [68]:
X.shape

(891, 198)

In [69]:
from sklearn.model_selection import train_test_split
# 評価用データを作成する。一旦HoldOut法で行う。
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [70]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
# モデルの構築をする。
def build_model(hid_dim=256,
                out_dim=1,
                hid_activation='relu',
                out_activation='sigmoid',
                optimizer='SGD',
                loss='binary_crossentropy',
                metrics=['accuracy'],
                drop_rate=0.1
                ):
  model = Sequential([
      Dense(hid_dim),
      Dropout(drop_rate),
      Activation(hid_activation),
      Dense(out_dim),
      Activation(out_activation),
  ])
  model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

  return model

# モデルを読み込む
model = build_model()

In [71]:
from sklearn.preprocessing import StandardScaler

sca = StandardScaler()
X_train_sca = sca.fit_transform(X_train)
X_val_sca = sca.transform(X_val)
X_test_sca = sca.transform(X_test)

In [72]:
# 学習
model.fit(X_train_sca, y_train, 
          batch_size=16,
          epochs=20,
          verbose=1,
          validation_data=(X_val_sca, y_val))

Epoch 1/20
45/45 [==============================] - 1s 5ms/step - loss: 0.6936 - accuracy: 0.6433 - val_loss: 0.6391 - val_accuracy: 0.6983
Epoch 2/20
45/45 [==============================] - 0s 2ms/step - loss: 0.5952 - accuracy: 0.7612 - val_loss: 0.5855 - val_accuracy: 0.7877
Epoch 3/20
45/45 [==============================] - 0s 2ms/step - loss: 0.5377 - accuracy: 0.8062 - val_loss: 0.5522 - val_accuracy: 0.7989
Epoch 4/20
45/45 [==============================] - 0s 3ms/step - loss: 0.5018 - accuracy: 0.8216 - val_loss: 0.5248 - val_accuracy: 0.7933
Epoch 5/20
45/45 [==============================] - 0s 2ms/step - loss: 0.4690 - accuracy: 0.8258 - val_loss: 0.5075 - val_accuracy: 0.7933
Epoch 6/20
45/45 [==============================] - 0s 3ms/step - loss: 0.4459 - accuracy: 0.8413 - val_loss: 0.4933 - val_accuracy: 0.7877
Epoch 7/20
45/45 [==============================] - 0s 2ms/step - loss: 0.4288 - accuracy: 0.8385 - val_loss: 0.4849 - val_accuracy: 0.7877
Epoch 8/20
45/45 [==

### valデータの評価

In [73]:
from sklearn.metrics import accuracy_score
# valデータを予測する。(スケーリングしたデータを使おう。)
val_probs = model.predict(X_val_sca)
# 形状を(179)にする
val_probs = val_probs.squeeze()
print(val_probs)

[0.15555125 0.14320901 0.13352561 0.8237741  0.74619824 0.8675711
 0.69100726 0.10827485 0.7201429  0.8459327  0.9022866  0.0902181
 0.4262549  0.1798228  0.13344106 0.83356524 0.25589055 0.69110703
 0.15535575 0.31112996 0.12439114 0.20724788 0.6016199  0.13418537
 0.11457542 0.10738128 0.24571642 0.14446062 0.12446111 0.59972554
 0.13613099 0.6338477  0.30352402 0.6034243  0.1388649  0.1676516
 0.28719747 0.69100726 0.86973923 0.1119177  0.14967066 0.09816149
 0.11197728 0.15345609 0.64579004 0.15157205 0.13626692 0.127083
 0.12403724 0.26292452 0.72492933 0.78801847 0.07122558 0.92901176
 0.09297499 0.8576747  0.14079925 0.3499589  0.7632434  0.7132024
 0.13094428 0.8002683  0.75161415 0.2180298  0.15345609 0.73800266
 0.2180365  0.11371925 0.21946236 0.9814964  0.7473546  0.98914754
 0.38963258 0.82108593 0.12648085 0.0927563  0.68950033 0.85371846
 0.7518063  0.530675   0.06764016 0.9847493  0.8376641  0.15344793
 0.271441   0.16584492 0.98163843 0.8648014  0.18376407 0.11166909
 

モデルの出力は(targetの数, 1)で出力される。
これを、以下のルールで変更する。 \
1. 出力の形状を(targetの数)に直す(np.squeeze()を用いる。)
2. thershold=0.5
3. 値がthresholdを下回ったら0
4. それ以上の場合は1 \

In [74]:
thershold = 0.5
# 予測を二値化する。
val_preds = np.where(val_probs < thershold, 0, 1)

Accuracyスコアで精度を計算する。

In [75]:
from sklearn.metrics import accuracy_score
# スコアを出力
print(accuracy_score(y_val, val_preds))

0.7821229050279329


### テストデータの予測とKaggleへの提出ファイルの作成。

In [76]:
# valデータと同じように予測
test_probs = model.predict(X_test_sca)
# test_probsの形状を直す。
test_probs = test_probs.squeeze()
# valデータと同じように予測を二値にする。
thershold = 0.5
test_preds = np.where(test_probs < thershold, 0, 1)

In [77]:
print(len(test_preds))
print(test_preds)

418
[0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 0
 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0
 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [78]:
if issubmit:
    os.makedirs('submit/', exist_ok=True)
    # submissionのサンプルファイルを読み込む
    submit = pd.read_csv('gender_submission.csv')
    submit['Survived'] = test_preds
    # 提出
    submit.to_csv('submit/my_submit3.csv', index=False)

### Cross Validationを作ろう

今回は、穴埋め形式ではなく一から実装してもらうことにする。 \
LightGBMのCross Validationのコードをコピペして、ニューラルネットワーク専用コードに編集すれば効率良く実装できる。(もちろん一から実装した方が勉強にはなる。)

In [80]:
from sklearn.model_selection import StratifiedKFold

FOLD = 5
# StratifiedKFoldのインスタンスを定義する。
skf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_probs = np.zeros(len(X_test))  # testデータの予測を格納するnumpy配列

# 各FOLDで学習を行う。
for i, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
  print(f'epoch{i+1}')
  # データを定義
  X_train, X_val = X.iloc[tr_idx], X.iloc[va_idx]
  y_train, y_val = y[tr_idx], y[va_idx]
  
  # データのスケーリングを行う。
  sca = StandardScaler()
  X_train_sca = sca.fit_transform(X_train)
  X_val_sca = sca.transform(X_val)
  X_test_sca = sca.transform(X_test)

  # 学習
  model.fit(X_train_sca, y_train, 
            batch_size=16,
            epochs=20,
            verbose=0,
            validation_data=(X_val_sca, y_val))
  
  # valデータを予測する。
  val_probs = model.predict(X_val_sca).squeeze()
  
  # 予測を二値化する。
  threshold = 0.5
  val_preds = np.where(val_probs < threshold, 0, 1)

  # スコアを出力
  print(accuracy_score(y_val, val_preds))

  # 予測結果をoof_predsに格納
  oof_preds[va_idx] = val_preds

  # testデータを予測しtest_probsに加算
  test_probs += model.predict(X_test_sca).squeeze()
  print('-'*40)


# test_probsをFOLD数で割る
test_probs /= FOLD
# 予測を二値化する
test_preds = np.where(test_probs < threshold, 0, 1)

# oofのスコアを算出
print('oof score : ', accuracy_score(y, oof_preds))

epoch1
0.7988826815642458
----------------------------------------
epoch2
0.8202247191011236
----------------------------------------
epoch3
0.8033707865168539
----------------------------------------
epoch4
0.8539325842696629
----------------------------------------
epoch5
0.8314606741573034
----------------------------------------
oof score :  0.8215488215488216


In [81]:
test_preds

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [82]:
if issubmit:
    os.makedirs('submit/', exist_ok=True)
    # submissionのサンプルファイルを読み込む
    submit = pd.read_csv('gender_submission.csv')
    submit['Survived'] = test_preds
    # 提出
    submit.to_csv('submit/my_submit4.csv', index=False)

## Pytorchで実装しよう

Pytorchは学術研究で広く用いられるフレームワークである。 \
実装を1からカスタマイズする必要があるので簡単な全結合ネットワークだとkerasの方が何かと便利だが、高度なディープラーニングなどで真価を発揮するため、実装してみる。

In [83]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Pytorchモデルを定義する。
class MLP(nn.Module):

  def __init__(self, input_dim, hid_dim=256, out_dim=1, drop_rate=0.1):
    # nn.Moduleクラスを継承する。
    super(MLP, self).__init__()
    # モデルのパーツを定義する。
    self.layer = nn.Sequential(
      nn.Linear(input_dim, hid_dim),
      nn.Dropout(drop_rate),
      nn.ReLU(),
      nn.Linear(hid_dim, out_dim),
      nn.Sigmoid()
    )

  def forward(self, x):
    # モデルの順伝搬を行う。
    return self.layer(x).squeeze()



In [84]:
# モデルの挙動を確認
sample_x = torch.randn([16, 198])
print(sample_x.size(1))
sample_model = MLP(sample_x.size(1))
sample_out = sample_model(sample_x)
print(sample_out.shape)

198
torch.Size([16])


In [85]:
# pytorchのデータセットを作る。
class MLPDataset(Dataset):

  def __init__(self, X, y=None, test=False):
    super().__init__()
    self.X = X
    self.test = test
    if not test:
      self.y = y

  def __len__(self): 
    return len(self.X)

  def __getitem__(self, idx):
    # X, yをtorch.Tensorに変換
    X_torch = torch.Tensor(self.X.values)
    if not self.test:
      y_torch = torch.Tensor(self.y)
      return [X_torch[idx], y_torch[idx]]
    else:
      return [X_torch[idx]]


In [86]:
# Datasetの動作確認
sample_dataset = MLPDataset(X, y)
sample_dataset[0]

[tensor([ 3.0000,  1.0000,  0.0000, 22.0000,  1.0000,  0.0000,  7.2500,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [87]:
# dataloaderを定義
sample_dataloader = DataLoader(sample_dataset, batch_size=16, shuffle=True)
for sd in sample_dataloader:
  print(sd[0].shape)
  print(sd[1].shape)
  break

torch.Size([16, 198])
torch.Size([16])


## Pytorchで学習

In [88]:
def torch_train(model, train_dataset, valid_dataset, criterion, optimizer,
                epochs=20, batch_size=16, verbose=0, threshold=0.5):
  # pytorchで学習を行う。
  for epoch in range(epochs):
    # 学習モードにする。
    model.train()
    # train_dataloaderを定義する。
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # train_lossを記録する。
    train_loss = 0.0
    for td in train_dataloader:
      # optimizerの勾配をリセット
      optimizer.zero_grad()
      # inputとtargetを定義
      input = td[0]
      target = td[1]
      output = model(input)
      # lossを求める。
      loss = criterion(output, target)
      train_loss += loss.item()
      # 逆伝搬
      loss.backward()
      # optimizerを更新
      optimizer.step()

    # 評価モードにする。
    model.eval()
    # valid_dataloderを定義する
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # outputとtargetを格納するリスト(accuracyの計算で使う。)
    targets = []
    outputs = []
    # lossを記録する。
    valid_loss = 0.0
    # 勾配計算をしないようにする。
    with torch.no_grad():
      for vd in valid_dataloader:
        input = vd[0]
        target = vd[1]
        output = model(input)
        loss = criterion(output, target)
        # target, output, lossを記録する。
        targets.extend(target.numpy())
        outputs.extend(output.numpy())
        valid_loss += loss.item()
    outputs = np.where(np.array(outputs) < threshold, 0, 1)
    valid_accuracy = accuracy_score(outputs, targets)
    if verbose != 0 and (epoch) % verbose == 0:
      print(f'epoch{epoch+1}')
      print("train loss : {}        valid_loss : {}       valid_accuracy : {}".format(train_loss, valid_loss, valid_accuracy))

  return model

def torch_predict(model, test_dataset, batch_size=16, threshold=0.5):
  # pytorchで学習したモデルを利用して予測を行う。
  model.eval()
  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
  test_probs = []  # 予測値を格納するリスト
  # 勾配計算をしないようにする。
  with torch.no_grad():
    for ted in test_dataloader:
      input = ted[0]
      output = model(input)
      test_probs.extend(output.numpy())
  test_preds = np.where(np.array(test_probs) < threshold, 0, 1)
  return test_preds

In [89]:
from torch.nn import BCELoss
from torch.optim import SGD, Adam
# ハイパーパラメータを設定
EPOCHS = 20
HID_DIM = 256
DROP_RATE = 0.1
BATCH_SIZE=16
feature_dim = X.shape[1]

# 評価用データを作成する。一旦HoldOut法で行う。
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# datasetを定義する。
train_dataset = MLPDataset(X_train, y_train)
valid_dataset = MLPDataset(X_val, y_val)
test_dataset = MLPDataset(X_test, test=True)

# モデルを定義する。
model = MLP(feature_dim, hid_dim=HID_DIM, drop_rate=DROP_RATE)
# 損失関数と最適化関数を定義する。
criterion = BCELoss()
optimizer = Adam(model.parameters(), lr=0.01)

# modelを学習する。
model = torch_train(model, train_dataset, valid_dataset, criterion, optimizer,
                    epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
# 予測する。
test_preds = torch_predict(model, test_dataset, batch_size=BATCH_SIZE)
print(test_preds)

epoch1
train loss : 29.052180230617523        valid_loss : 6.094848453998566       valid_accuracy : 0.776536312849162
epoch2
train loss : 25.014594167470932        valid_loss : 6.225804626941681       valid_accuracy : 0.770949720670391
epoch3
train loss : 23.061429053544998        valid_loss : 5.970846593379974       valid_accuracy : 0.7821229050279329
epoch4
train loss : 21.693679824471474        valid_loss : 5.787984237074852       valid_accuracy : 0.8212290502793296
epoch5
train loss : 21.842391073703766        valid_loss : 5.457716062664986       valid_accuracy : 0.776536312849162
epoch6
train loss : 20.3119033575058        valid_loss : 5.897383958101273       valid_accuracy : 0.8044692737430168
epoch7
train loss : 20.321448355913162        valid_loss : 5.5431570410728455       valid_accuracy : 0.770949720670391
epoch8
train loss : 20.3828115016222        valid_loss : 6.00776469707489       valid_accuracy : 0.770949720670391
epoch9
train loss : 19.623865351080894        valid_loss 

In [90]:
if issubmit:
    os.makedirs('submit/', exist_ok=True)
    # submissionのサンプルファイルを読み込む
    submit = pd.read_csv('gender_submission.csv')
    # サンプルファイルのtargetカラムをtest_predsに置き換える。
    submit['Survived'] = test_preds
    # 提出
    submit.to_csv('submit/my_submit5.csv', index=False)

## Cross Validationを実装する。

pytorchの学習と予測は独自で、関数化してあるため実装は、そこまで難しくないはずである。

In [91]:
from sklearn.model_selection import StratifiedKFold

# ハイパーパラメータを設定
EPOCHS = 20
HID_DIM = 256
DROP_RATE = 0.1
BATCH_SIZE=16
feature_dim = X.shape[1]

FOLD = 5
# StratifiedKFoldのインスタンスを定義する。
skf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_probs = np.zeros(len(X_test))  # testデータの予測を格納するnumpy配列

# 各FOLDで学習を行う。
for i, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
  print(f'fold{i+1}')
  # データを定義
  X_train, X_val = X.iloc[tr_idx], X.iloc[va_idx]
  y_train, y_val = y[tr_idx], y[va_idx]
  
  # データのスケーリングを行う。
  sca = StandardScaler()
  X_train_sca = sca.fit_transform(X_train)
  X_val_sca = sca.transform(X_val)
  X_test_sca = sca.transform(X_test)

  # データセットを作成する。
  train_dataset = MLPDataset(X_train, y_train)
  valid_dataset = MLPDataset(X_val, y_val)
  test_dataset = MLPDataset(X_test, test=True)

  # モデルを定義する。
  model = MLP(feature_dim, hid_dim=HID_DIM, drop_rate=DROP_RATE)
  # 損失関数と最適化関数を定義する。
  criterion = BCELoss()
  optimizer = Adam(model.parameters(), lr=0.01)

  # 学習
  model = torch_train(model, train_dataset,
                      valid_dataset, 
                      criterion,
                      optimizer,
                      epochs=EPOCHS, 
                      batch_size=BATCH_SIZE,
                      verbose=0)
  
  # valデータを予測する。
  threshold = 0.5
  val_preds = torch_predict(model, valid_dataset,
                            batch_size=BATCH_SIZE,
                            threshold=threshold)

  # スコアを出力
  print(accuracy_score(y_val, val_preds))

  # 予測結果をoof_predsに格納
  oof_preds[va_idx] = val_preds

  # testデータを予測しtest_probsに加算
  test_probs += torch_predict(model, test_dataset,
                            batch_size=BATCH_SIZE,
                            threshold=threshold)
  print('-'*40)

# test_probsをFOLD数で割る
test_probs /= FOLD
# 予測を二値化する
test_preds = np.where(test_probs < threshold, 0, 1)

# oofのスコアを算出
print('oof score : ', accuracy_score(y, oof_preds))

fold1
0.770949720670391
----------------------------------------
fold2
0.8146067415730337
----------------------------------------
fold3
0.8033707865168539
----------------------------------------
fold4
0.7808988764044944
----------------------------------------
fold5
0.8089887640449438
----------------------------------------
oof score :  0.7957351290684624


In [92]:
if issubmit:
    os.makedirs('submit/', exist_ok=True)
    # submissionのサンプルファイルを読み込む
    submit = pd.read_csv('gender_submission.csv')
    # サンプルファイルのtargetカラムをtest_predsに置き換える。
    submit['Survived'] = test_preds
    # 提出
    submit.to_csv('submit/my_submit6.csv', index=False)

## 参考サイト
- Kerasの公式ドキュメント : https://keras.io/ja/
- Pytorchの公式ドキュメント : https://pytorch.org/docs/stable/index.html
- Pytorchでテーブルデータを処理する方法 : https://towardsdatascience.com/deep-learning-using-pytorch-for-tabular-data-c68017d8b480